In [2]:
import json
import pymongo
import pandas as pd

In [3]:
# log in db
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["dataset"]
mycol = mydb["TR_CH"]

# read TR file
file_path = "transaction_CH.json"
with open(file_path,"rb")as f:
    data = json.load(f)

In [4]:
data = pd.DataFrame(data)

In [5]:
data.columns

Index(['address_code', 'branch_id', 'branch_name', 'category', 'enabled_at',
       'gender', 'id', 'invoice_datetime', 'invoice_number', 'invoice_price',
       'is_member_prepaid', 'is_taiwan', 'lat', 'level_expiry_date', 'lng',
       'points_gained', 'status', 'store_id', 'transaction_amount',
       'transaction_id', 'user_id', 'user_level', 'invoice_year',
       'invoice_month', 'invoice_day', 'invoice_weekday'],
      dtype='object')

In [6]:
data.head()

,address_code,branch_id,branch_name,category,enabled_at,gender,id,invoice_datetime,invoice_number,invoice_price,...,status,store_id,transaction_amount,transaction_id,user_id,user_level,invoice_year,invoice_month,invoice_day,invoice_weekday
0,內湖區,005,微風南京,主題餐廳,2016-10-25 21:12:32,female,1964289,2017-01-11 19:20:19,SE51702683,380.0,...,success,02080,380.0,4777778,275697,C,2017,1,11,Wed
1,內湖區,005,微風南京,化妝品/保養品/香氛,2016-10-25 21:12:32,female,2002788,2017-01-17 19:13:35,SE51860662,650.0,...,success,10160,650.0,4857769,275697,C,2017,1,17,Tue
2,內湖區,005,微風南京,化妝品/保養品/香氛,2016-10-25 21:12:32,female,2002794,2017-01-17 19:14:25,SE51860663,650.0,...,success,10160,650.0,4857781,275697,C,2017,1,17,Tue
3,內湖區,001,微風廣場,超市,2016-10-25 21:12:32,female,2052065,2017-01-25 17:31:15,SF58883581,90.0,...,success,19050,90.0,4959439,275697,C,2017,1,25,Wed
4,內湖區,009,微風信義,餐廳,2016-10-25 21:12:32,female,2073013,2017-01-28 19:06:37,SE73879788,528.0,...,success,40010,528.0,5002536,275697,C,2017,1,28,Sat


In [7]:
data.drop(columns=["branch_id","enabled_at","id","invoice_datetime","invoice_number",
                   "lat","store_id","transaction_id","user_id","level_expiry_date","transaction_amount","lng"],inplace=True)

#data.drop(columns=["transaction_amount","lng"],inplace=True)

In [13]:
data.dropna(axis=0,how="any")
data = data[data.gender != "nan"]
data = data[data.address_code != "nan"]
data = data[data.address_code != ""]
data = data[data.category != ""]

In [9]:
data["is_member_prepaid"] = data["is_member_prepaid"].replace({"0":"F","1":"T"})

In [10]:
data = data.astype({
    "invoice_price":"float64",
    "points_gained":"float64",
})

data = data.astype({
    "invoice_month":"string",
    "invoice_day":"string",
})

In [14]:
parsed_js = json.loads(data.to_json(orient = 'records',force_ascii=False))    
mycol.insert_many(parsed_js)